### Russian Language Toxic Comments
https://www.kaggle.com/blackmoon/russian-language-toxic-comments  
Small dataset with labeled comments from 2ch.hk and pikabu.ru

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.options.display.max_columns = None
np.set_printoptions(suppress=True)

import warnings
warnings.filterwarnings('ignore')

In [18]:
df = pd.read_csv('./datasets/toxic_labeled.csv')
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [20]:
df.toxic.value_counts(normalize=True)

0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64

In [31]:
df.shape

(14412, 2)

In [24]:
!pip install gensim

In [25]:
from gensim.corpora.dictionary import Dictionary

In [26]:
import re
import numpy as np
from nltk.corpus import stopwords

In [27]:
from razdel import tokenize
import pymorphy2

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AVasilev\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [28]:
stopword_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [30]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [12]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub("n", ' ', text)

    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    #print(tokens)
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
                #print(temp_cach)
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
                #print(w,' : ',temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    #print(words_lem_without_stopwords)
    return words_lem_without_stopwords

In [37]:
%%time
df['comment'] = df['comment'].apply(lambda x: clean_text(x), 1)

Wall time: 817 ms


In [38]:
%%time
df['comment'] = df['comment'].apply(lambda x: lemmatization(x), 1)

Wall time: 14.8 s


In [39]:
df.head()

,comment,toxic
0,"[верблюдовто, дебил, бл]",1.0
1,"[хохол, это, отдушина, затюканый, россиянин, м...",1.0
2,"[собака, собачий, смерть]",1.0
3,"[страница, обновить, дебил, это, оскорбление, ...",1.0
4,"[убедить, страничный, пдф, скрипаль, отравить,...",1.0


In [40]:
df.tail()

,comment,toxic
14407,"[вонючий, совковый, скот, прибежать, ныть, сто...",1.0
14408,"[любить, гоблин, тупорылый, чтоль, какуюнибудь...",1.0
14409,"[посмотреть, утомлённый, солнце, оказаться, эт...",0.0
14410,"[крымотред, нарушать, правило, раздел, тк, нем...",1.0
14411,"[сей, пора, пересматривать, видео, орамбо, кст...",0.0


In [44]:
#сформируем список наших текстов, разбив еще и на пробелы
comments = [t for t in df['comment'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(comments)
common_corpus = [common_dictionary.doc2bow(comment) for comment in comments]

In [46]:
len(common_dictionary)

35543

In [48]:
common_dictionary.id2token

{0: 'бл',
 1: 'верблюдовто',
 2: 'дебил',
 3: 'вон',
 4: 'ещё',
 5: 'затюканый',
 6: 'кисель',
 7: 'мол',
 8: 'отдушина',
 9: 'плохой',
 10: 'придумать',
 11: 'россиянин',
 12: 'хохлов',
 13: 'хохол',
 14: 'это',
 15: 'смерть',
 16: 'собака',
 17: 'собачий',
 18: 'верить',
 19: 'воображать',
 20: 'доказать',
 21: 'друг',
 22: 'множественный',
 23: 'недебил',
 24: 'обновить',
 25: 'оскорбление',
 26: 'писать',
 27: 'страница',
 28: 'твой',
 29: 'факт',
 30: 'число',
 31: 'анализировать',
 32: 'ватник',
 33: 'думать',
 34: 'отравить',
 35: 'пдф',
 36: 'пытаться',
 37: 'россия',
 38: 'скрипаль',
 39: 'страничный',
 40: 'убедить',
 41: 'здравоохранение',
 42: 'зимбабве',
 43: 'рф',
 44: 'система',
 45: 'современный',
 46: 'стан',
 47: 'тупой',
 48: 'эталон',
 49: 'являться',
 50: 'абсолютный',
 51: 'борд',
 52: 'брипидор',
 53: 'дно',
 54: 'заменить',
 55: 'игнорировать',
 56: 'инф',
 57: 'лишить',
 58: 'марвести',
 59: 'модератор',
 60: 'неадекват',
 61: 'неадекватность',
 62: 'недостаточ

In [49]:
common_dictionary.doc2bow(df.iloc[0,0], allow_update=True)

[(0, 1), (1, 1), (2, 1)]

In [83]:
X = df.comment
y = df.toxic

In [ ]:
# Формирую из списков слов строки
X = X.apply(lambda x: ' '.join(x))

In [66]:
from sklearn.model_selection import train_test_split

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [90]:
X_train.iloc[0, ]

'единый диспетчерская служба везде'

In [61]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import xgboost as xgb

In [110]:
pipeline = Pipeline([ 
                     ('doc_tfidf', TfidfVectorizer(stop_words=stopword_ru, max_features= 1000)), 
                     ('clf', xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss'))
                    ])

In [111]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('doc_tfidf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=['и', 'в', 'во', 'не', 'что', 'он',
                                             'на', 'я', 'с', 'со', 'как', 'а',
                                             'то', 'все', 'она', 'так', 'его',
                                             'но', 'да', 'ты', 'к', 'у', 'же',
                                             'вы', 'за', 'бы', 'по', 'только',
                                             'ее', 'мне', ...])),
                ('clf',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eva...
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                           

In [112]:
y_preds = pipeline.predict(X_test)

In [113]:
from sklearn.metrics import roc_auc_score, classification_report

In [114]:
roc_auc_score(y_test, y_preds)

0.7149130683368004

In [116]:
target_names = ['nontoxic', 'toxic']
print(classification_report(y_test, y_preds, target_names=target_names))

              precision    recall  f1-score   support

    nontoxic       0.79      0.96      0.86      2906
       toxic       0.84      0.47      0.61      1418

    accuracy                           0.80      4324
   macro avg       0.82      0.71      0.74      4324
weighted avg       0.81      0.80      0.78      4324



In [117]:
from sklearn.model_selection import GridSearchCV

params={'clf__n_estimators':[100, 150, 200, 250, 300],
        'clf__learning_rate':[0.1, 0.05, 0.01],
        'clf__max_depth':[3, 5, 7, 10, 15, 20],
        'clf__subsample': [0.5, 0.7, 0.9, 1],
        'clf__tree_method': ['gpu_hist']
        }

In [118]:
grids = GridSearchCV(pipeline,
                    param_grid=params,
                    cv=10,
                    refit=False)

In [119]:
%%time
search = grids.fit(X_train, y_train)

Wall time: 9h 37min 5s


In [120]:
search.best_params_

{'clf__learning_rate': 0.1,
 'clf__max_depth': 5,
 'clf__n_estimators': 300,
 'clf__subsample': 0.5,
 'clf__tree_method': 'gpu_hist'}

In [121]:
search.best_score_

0.7296813794893577

In [123]:
final_pipeline = Pipeline([ 
                     ('doc_tfidf', TfidfVectorizer(stop_words=stopword_ru, max_features= 1000)), 
                     ('clf', xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss',
                                              learning_rate=0.1, max_depth=5, n_estimators=300,
                                              subsample=0.5, tree_method='gpu_hist'))
                    ])

In [124]:
final_pipeline.fit(X, y)

Pipeline(steps=[('doc_tfidf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=['и', 'в', 'во', 'не', 'что', 'он',
                                             'на', 'я', 'с', 'со', 'как', 'а',
                                             'то', 'все', 'она', 'так', 'его',
                                             'но', 'да', 'ты', 'к', 'у', 'же',
                                             'вы', 'за', 'бы', 'по', 'только',
                                             'ее', 'мне', ...])),
                ('clf',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eva...
                               gamma=0, gpu_id=0, importance_type='gain',
                               interaction_constraints='', learning_rate=0.1,
                               max_delta_step=0, max_depth=5,
                       

In [125]:
from sklearn.model_selection import cross_validate

In [130]:
%%time
score = cross_validate(final_pipeline, X, y, scoring=['roc_auc', 'f1_weighted'], n_jobs=-1)

Wall time: 20 s


In [131]:
score

{'fit_time': array([18.23399901, 18.50601912, 18.65699816, 17.89799738, 18.74299836]),
 'score_time': array([0.14499879, 0.12697744, 0.14399934, 0.11699915, 0.15299892]),
 'test_roc_auc': array([0.69878773, 0.82613068, 0.80631005, 0.73637538, 0.78289831]),
 'test_f1_weighted': array([0.70198968, 0.75751229, 0.72411972, 0.6374402 , 0.70847016])}

In [132]:
score['test_roc_auc'].mean()

0.7701004330279135

In [133]:
score['test_f1_weighted'].mean()

0.7059064095978913